#  UNIVARIATE METHOD 

This file is a port-over from Janelcy Alferes' Matlab scripts

This script contains the different steps to follow to apply the univariate
method for on-line data treatment.  It's a type script with the different
step for an example of parameter X of a Sensor.

## Importing the required libraries 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

from pandas.plotting import register_matplotlib_converters

import Sensors
import FaultDetection
import ModelCalibration
import OutlierDetection
import outlierdetection_Online_EWMA
import PlottingTools
import Smoother
import TreatedData
import DataCoherence
from DefaultSettings import DefaultParam

register_matplotlib_converters()


## Raw data  

In [ ]:
path = '../sample_data/influent3.csv'
raw_data = pd.read_csv(path, sep=';')
raw_data.datetime = pd.to_datetime(raw_data.datetime)
raw_data.set_index('datetime', inplace=True, drop=True)
data = raw_data['15-09-2017 00:00:00':'15-04-2018 00:00:00']


## Create a 'Sensor' object to store the data and the metadata

In [ ]:
sensors = Sensors.parse_dataframe(data)

## Plot raw data

In [ ]:
title = 'Pre-processed Data'
PlottingTools.plotRaw_D_mpl(data)

### Choose the time series to filter

In [ ]:
sensor = sensors[0]
channel = sensor.channels['COD']  # Variable to be filtered

T0 = channel.start
TF = channel.end

## Generate default parameters for the filter

In [ ]:
channel.params = DefaultParam()

Example: here's how to manually set one of the filter's parameters

In [ ]:
channel.params['outlier_detection']['nb_reject'] = 100

## Select a subset of the data for calibration

In [ ]:
Tini = '15 January 2018'
Tfin = '15 February 2018'

channel.calib = {'start': Tini, 'end': Tfin}
start = channel.calib['start']
end = channel.calib['end']
# Plot calibration data
title = 'Calibration subset'

calib_data = channel.raw_data[start:end]

PlottingTools.plotUnivar_plotly(channel)

## Test the dataset for missing values, NaN, etc.

In [ ]:
flag = DataCoherence.data_coherence(channel)
flag

## fix issues with the calibration data

In [ ]:
channel = DataCoherence.resample(channel, '2 min')
channel = DataCoherence.fillna(channel)

## Try the data coherence check again

In [ ]:
flag = DataCoherence.data_coherence(channel)
flag

## OK, we're good to go!


## Outlier detection

In [ ]:
filtration_method = "Online_EWMA"
channel.params['outlier_detection']['method'] = filtration_method

channel = OutlierDetection.outlier_detection(channel)

In [ ]:
PlottingTools.plotOutliers_plotly(channel)


##  Data smoother

In [ ]:
# Set parameters
channel.params['data_smoother']['h_smoother'] = 10

channel = Smoother.kernel_smoother(channel)

In [ ]:
PlottingTools.plotOutliers_plotly(channel)

## Fault detection

In [ ]:
# minimum real expected value of the variable
channel.params['fault_detection_uni']['range_min'] = 50

# maximum real expected value of the variable
channel.params['fault_detection_uni']['range_max'] = 550

# Definition limit of scores:
channel.params['fault_detection_uni']['corr_min'] = -50
channel.params['fault_detection_uni']['corr_max'] = 50

# minimum expected slope based on good data series
channel.params['fault_detection_uni']['slope_min'] = -10

# maximum expected slope based on a good data series
channel.params['fault_detection_uni']['slope_max'] = 10

# Minimum variation between accepted data and smoothed data
channel.params['fault_detection_uni']['std_min'] = -10

# Maximum variation between accepted data and smoothed data
channel.params['fault_detection_uni']['std_max'] = 10


In [ ]:
# Calcul Q_corr, Q_std, Q_slope, Q_range:
channel = FaultDetection.D_score(channel)


##  Treated data

In [ ]:
# To see the treated data and the deleted data:
channel = TreatedData.TreatedD(channel)

In [ ]:
# plot the raw data and treated data:
PlottingTools.plotTreatedD_plotly(channel)

In [ ]:
# Percentage of outliers and deleted data
filtration_method = channel.info['current_filtration_method']
print(channel.info['filtration_results'][filtration_method])